In [ ]:
# General notebook settings
import warnings

warnings.filterwarnings("error", category=DeprecationWarning)

# SciGRID Network

In this example, the dispatch of generators is optimised using the linear optimisation, then a non-linear power flow is run on the resulting dispatch.
This example covers the German power system (roughly in the mid 2010s) for a single day at hourly resolution.

## Data sources

- **Grid:** Based on [SciGRID](https://openenergyplatform.org/factsheets/models/22/) which is based on [OpenStreetMap](http://www.openstreetmap.org/).

- **Load size and location:** Distributed proportional to NUTS3 GDP and population. In doubt, load and generation is attached to the 220kV substation.

- **Load time series:** From ENTSO-E hourly data, scaled up uniformly by factor 1.12 (a simplification of the methodology in [Schumacher and Hirth (2015)](https://ideas.repec.org/p/fem/femwpa/2015.88.html)).

- **Conventional power plant capacities and locations:** Based on list of [BNetzA](https://www.bnetza.de/) (German Federal Network Agency).

- **Wind and solar capacities and locations:** Based on [EEG Stammdaten](https://www.netztransparenz.de/de-de/Erneuerbare-Energien-und-Umlagen/EEG/EEG-Abrechnungen/EEG-Jahresabrechnungen/EEG-Anlagenstammdaten), [EnergyMap](http://www.energymap.info/download.html), which represents capacities at the end of 2014.

- **Wind and solar time series:** Generated with REatlas tool based on [Andresen et al. (2015)](https://doi.org/10.1016/j.energy.2015.09.071).

## Warnings

**The data behind this example is no longer supported or updated. The dataset is only intended to demonstrate the capabilities of PyPSA and is not suitable for research purposes. Have a look at [PyPSA-Eur](https://github.com/PyPSA/pypsa-eur) for a newer grid model with active support.**

Known problems include:

1. Rough approximations have been made for missing grid data, e.g. 220kV-380kV transformers and connections between close sub-stations missing from OSM.

2. There appears to be some unexpected congestion in parts of the network, which may mean for example that the load attachment method is inaccurate, particularly in regions with a high density of substations.

3. Attaching power plants to the nearest high voltage substation may not reflect reality.

4. The borders and neighbouring countries are not represented.

5. Hydroelectric power stations are not modelled accurately.

6. The marginal costs of generators are illustrative, not accurate.

7. Only the first day of 2011 is in the example, which is not representative.

8. The ENTSO-E total load for Germany may not be scaled correctly.

9. Biomass from the EEG reference data (Stammdaten) are not read in.

10. Power plant start up costs, ramping limits, and minimum part loads are not considered.

## Input Data

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pypsa

n = pypsa.examples.scigrid_de()

Plot the distribution of the load and of generating tech

In [ ]:
fig, ax = plt.subplots(
    1,
    1,
    subplot_kw={"projection": ccrs.EqualEarth()},
)

load_distribution = n.loads_t.p_set.loc[n.snapshots[0]].groupby(n.loads.bus).sum()
n.plot(bus_size=load_distribution / 30000, ax=ax, title="Load distribution");

In [ ]:
n.generators.groupby("carrier")["p_nom"].sum().round(1)

In [ ]:
n.storage_units.groupby("carrier")["p_nom"].sum().round(1)

In [ ]:
techs = ["Gas", "Brown Coal", "Hard Coal", "Wind Offshore", "Wind Onshore", "Solar"]

n_graphs = len(techs)
n_cols = 3
if n_graphs % n_cols == 0:
    n_rows = n_graphs // n_cols
else:
    n_rows = n_graphs // n_cols + 1


fig, axes = plt.subplots(
    nrows=n_rows, ncols=n_cols, subplot_kw={"projection": ccrs.EqualEarth()}
)
size = 6
fig.set_size_inches(size * n_cols, size * n_rows)

for i, tech in enumerate(techs):
    i_row = i // n_cols
    i_col = i % n_cols

    ax = axes[i_row, i_col]
    gens = n.generators[n.generators.carrier == tech]
    gen_distribution = (
        gens.groupby("bus").sum()["p_nom"].reindex(n.buses.index, fill_value=0)
    )
    n.plot(ax=ax, bus_size=gen_distribution / 20000)
    ax.set_title(tech)
fig.tight_layout()

## Rolling-Horizon Optimisation

Run optimisation (linear optimal power flow) on the first day of 2011.

To approximate $N-1$ security and allow room for reactive power flows, we do not allow any line to be loaded above 70% of their thermal rating:

In [ ]:
contingency_factor = 0.7
n.lines.s_max_pu = contingency_factor

There are some infeasibilities without small extensions

In [ ]:
n.lines.loc[["316", "527", "602"], "s_nom"] = 1715

We are performing the optimisation for one day, 4 snapshots at a time.

In [ ]:
n.optimize.optimize_with_rolling_horizon(horizon=4, overlap=0, log_to_console=False);

## Plot dispatch time series

In [ ]:
p_by_carrier = n.generators_t.p.T.groupby(n.generators.carrier).sum().T
to_drop = p_by_carrier.max()[p_by_carrier.max() < 1700].index
p_by_carrier.drop(to_drop, axis=1, inplace=True)
p_by_carrier.columns

In [ ]:
colors = {
    "Brown Coal": "brown",
    "Hard Coal": "k",
    "Nuclear": "r",
    "Run of River": "green",
    "Wind Onshore": "blue",
    "Solar": "yellow",
    "Wind Offshore": "cyan",
    "Waste": "orange",
    "Gas": "orange",
}
# reorder
cols = [
    "Nuclear",
    "Run of River",
    "Brown Coal",
    "Hard Coal",
    "Gas",
    "Wind Offshore",
    "Wind Onshore",
    "Solar",
]
p_by_carrier = p_by_carrier[cols]

In [ ]:
c = [colors[col] for col in p_by_carrier.columns]
fig, ax = plt.subplots()
p_by_carrier.div(1e3).plot(kind="area", ax=ax, lw=0, color=c, alpha=0.7)
ax.legend(ncol=3, loc="upper left", bbox_to_anchor=(0, 1.02, 1, 0.2), frameon=False)
ax.set_ylabel("GW")
ax.set_xlabel("")

## Plot storage time series

In [ ]:
fig, ax = plt.subplots()

p_storage = n.storage_units_t.p.sum(axis=1)
state_of_charge = n.storage_units_t.state_of_charge.sum(axis=1)
p_storage.plot(label="Pumped hydro dispatch", ax=ax)
state_of_charge.plot(label="State of charge", ax=ax)

ax.axhline(0, color="k", lw=0.5, ls="--")
ax.legend()
ax.set_ylabel("MWh")
ax.set_xlabel("")

## Line loading from optimisation

With the linear power flow, there is the following per unit loading at 3 AM:

In [ ]:
now = n.snapshots[4]
loading = n.lines_t.p0.loc[now] / n.lines.s_nom
loading.describe()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.EqualEarth()})
n.plot(
    ax=ax,
    line_color=loading.abs(),
    line_cmap="viridis",
    title="Line loading",
    bus_size=1e-3,
);

## Locational marginal prices

Let's have a look at the distribution of marginal prices:

In [ ]:
n.buses_t.marginal_price.loc[now].describe()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})

plt.hexbin(
    n.buses.x,
    n.buses.y,
    gridsize=20,
    C=n.buses_t.marginal_price.loc[now],
    cmap="viridis",
    zorder=-1,
)
n.plot(ax=ax, line_width=1, bus_size=0)

cb = plt.colorbar(location="right")
cb.set_label("Locational Marginal Price (€/MWh)")

## Curtailment

By considering how much power is available and how much is generated, you can see what share is curtailed: 

In [ ]:
carrier = "Wind Onshore"

capacity = n.generators.groupby("carrier").sum().at[carrier, "p_nom"]
p_available = n.generators_t.p_max_pu.multiply(n.generators["p_nom"])
p_available_by_carrier = p_available.T.groupby(n.generators.carrier).sum().T
p_curtailed_by_carrier = p_available_by_carrier - p_by_carrier

In [ ]:
p_df = pd.DataFrame(
    {
        carrier + " available": p_available_by_carrier[carrier],
        carrier + " dispatched": p_by_carrier[carrier],
        carrier + " curtailed": p_curtailed_by_carrier[carrier],
    }
)

p_df[carrier + " capacity"] = capacity

In [ ]:
p_df.loc[p_df["Wind Onshore curtailed"] < 0, "Wind Onshore curtailed"] = 0

In [ ]:
fig, ax = plt.subplots()
p_df[[carrier + " dispatched", carrier + " curtailed"]].plot(kind="area", ax=ax, lw=0)
p_df[[carrier + " available", carrier + " capacity"]].plot(ax=ax)

ax.set_xlabel("")
ax.set_ylabel("Power [MW]")
ax.set_ylim([0, 40_000])
ax.legend()

## Non-Linear Power Flow

Now perform a full Newton-Raphson power flow on the first day. For the power flow, provide the setpoints from the optimisation.

In [ ]:
n.optimize.fix_optimal_dispatch()

Set nearly all buses to PV, since we don't know what Q set points are. But we needs some PQ buses to ensure that the Jacobian is not singular.:

In [ ]:
n.generators.control = "PV"

f = n.generators[n.generators.bus == "492"]
n.generators.loc[f.index, "control"] = "PQ"

Now, perform the non-linear PF.

In [ ]:
info = n.pf();

Any failed to converge?

In [ ]:
(~info.converged).any().any()

With the non-linear load flow, there is the following per unit loading of the full thermal rating.

In [ ]:
(n.lines_t.p0.loc[now] / n.lines.s_nom).describe()

Let's inspect the voltage angle differences across the lines have (in degrees)

In [ ]:
df = n.lines.copy()

for b in ["bus0", "bus1"]:
    df = pd.merge(
        df, n.buses_t.v_ang.loc[[now]].T, how="left", left_on=b, right_index=True
    )

s = df[str(now) + "_x"] - df[str(now) + "_y"]

(s * 180 / np.pi).describe()

Plot the reactive power

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.EqualEarth()})

q = n.buses_t.q.loc[now]
bus_color = q.map(lambda x: "b" if x < 0 else "r")

n.plot(
    bus_size=q.abs() / 1e4,
    ax=ax,
    bus_color=bus_color,
    title="Reactive power feed-in\n(red = positive, blue = negative)",
);